In [2]:
import openai 
import numpy as np
import pandas as pd
import tiktoken
import tqdm
import glob
import json
import re

In [3]:
df = pd.read_csv('../dati_rivista/totale_isr_palestine.csv')
#dataframe usato per annotare con GPT. Selezionati tutti gli articoli con nel titolo "palestin|israel"

In [4]:
df = df[df['titolo'].str.contains('israel|palestin',flags=re.I, regex=True).fillna(False)]

In [8]:
from openai import OpenAI
client = OpenAI(api_key='YOUR_API_KEY')

# Count tokens

In [5]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


# Classificazione binaria

In [6]:
df_benchmark = pd.read_csv('../test_articles.csv')

In [7]:
df_benchmark = df_benchmark.rename(columns={'bias':'bias_true'})

In [8]:
df_benchmark['pred_label'] = [None]*len(df_benchmark)

In [9]:
from sklearn.metrics import accuracy_score

In [10]:
df_anti_palestina = df_benchmark[df_benchmark['bias_true']=='anti_palestina']

In [11]:
sample_size = len(df_anti_palestina)

In [12]:
sample_size

14

In [13]:
df_non_anti_palestina = df_benchmark[df_benchmark['bias_true']!='anti_palestina'].sample(sample_size)

In [14]:
df_non_anti_palestina['bias_true'] = ['no_bias']*len(df_non_anti_palestina)

In [15]:
df_binary = pd.concat([df_anti_palestina,df_non_anti_palestina]).sample(frac=1)

In [16]:
sistema_binary = '''Sei un esperto giornalista internazionale, molto imparziale e obiettivo. Stai effettuando una ricerca per un articolo sulla copertura mediatica del conflitto israelo-palestinese. Hai raccolto un campione di articoli da diversi giornali e ne stai leggendo gli articoli per capire se c'è un bias nella copertura mediatica. Il tuo compito è classificare gli articoli in due categorie:
- categoria pro_israele: articoli con un bias a favore di Israele, che vogliono giustificare le violenze di Israele senza considerare la complessità del contesto.
- categoria no_bias: articoli che non mostrano un bias a favore di Israele o che giustificano le azioni di Hamas.

Fornisci le risposte solo in formato Json con due proprietà: reason (500 characters), bias (pro_israele, no_bias). Dopo la fine del json, stampa uu.'''

In [17]:
df_binary['bias_true'] = df_binary['bias_true'].replace('anti_palestina','pro_israele')

In [18]:
count = 0
for i in tqdm.tqdm(range(len(df_binary))):
    text = df_binary.iloc[i,0]

    messages=[{"role": "system", "content": sistema_binary},
    {"role": "user", "content": "Medio Oriente, Salvini: 'Reazione Israele inevitabile dopo carneficina'"},
    {"role":"assistant","content": '{"reason": "Le azioni di Israele vengono presentate esclusivamente come una reazione giustificata.", "bias":"pro_israele"}uu'},
    {"role": "user", "content": "Saltata la tregua Israele riprende a colpire la Striscia  Ma si tratta ancora"},
    {"role":"assistant", "content":'{"reason": "Il titolo cita entrambe le parti in gioco, senza prendere una posizione e senza nascondere nulla.", "bias":"no_bias"}uu'},
    #{"role":"user", "content":"I palestinesi di Hamas sono come i partigiani"},
    #{"role":"assistant","content":'{"reason":"Il confronto tra i palestinesi di Hamas e i partigiani può essere interpretato come un tentativo di idealizzare o giustificare le azioni di Hamas.", "bias":"no_bias"}uu'},
    {"role":'user', "content":text}]
    
    #print(text)

    count+=num_tokens_from_messages(messages)

100%|██████████| 28/28 [00:00<00:00, 80.76it/s]


In [19]:
count

11126

## one test

In [23]:
text = df_binary.iloc[2,0]

In [24]:
messages=[{"role": "system", "content": sistema_binary},
    {"role": "user", "content": "Medio Oriente, Salvini: 'Reazione Israele inevitabile dopo carneficina'"},
    {"role":"assistant","content": '{"reason": "Le azioni di Israele vengono presentate esclusivamente come una reazione giustificata.", "bias":"pro_israele"}uu'},
    {"role": "user", "content": "Saltata la tregua Israele riprende a colpire la Striscia  Ma si tratta ancora"},
    {"role":"assistant", "content":'{"reason": "Il titolo cita entrambe le parti in gioco, senza prendere una posizione e senza nascondere nulla.", "bias":"no_bias"}uu'},
    #{"role":"user", "content":"I palestinesi di Hamas sono come i partigiani"},
    #{"role":"assistant","content":'{"reason":"Il confronto tra i palestinesi di Hamas e i partigiani può essere interpretato come un tentativo di idealizzare o giustificare le azioni di Hamas.", "bias":"no_bias"}uu'},
    {"role":'user', "content":text}]

In [25]:
text

'Italiani solidali con Israele ma la maggioranza boccia l’invasione di Gaza'

In [34]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0
    )

In [35]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"reason": "L\'articolo sembra concentrarsi su un singolo incidente per dipingere in modo negativo i sostenitori palestinesi, senza considerare il contesto più ampio del conflitto.", "bias":"pro_israele"}uu', role='assistant', function_call=None, tool_calls=None))

## Predizioni con OpenAI per tutto il benchmark

In [36]:
start = 0
end = len(df_binary)

In [37]:
pred_raw = []

for i in tqdm.tqdm(range(start,end)):
    text = df_binary.iloc[i,0]

    messages=[{"role": "system", "content": sistema_binary},
    {"role": "user", "content": "Medio Oriente, Salvini: 'Reazione Israele inevitabile dopo carneficina'"},
    {"role":"assistant","content": '{"reason": "Le azioni di Israele vengono presentate esclusivamente come una reazione giustificata.", "bias":"pro_israele"}uu'},
    {"role": "user", "content": "Saltata la tregua Israele riprende a colpire la Striscia  Ma si tratta ancora"},
    {"role":"assistant", "content":'{"reason": "Il titolo cita entrambe le parti in gioco, senza prendere una posizione e senza nascondere nulla.", "bias":"no_bias"}uu'},
    #{"role":"user", "content":"I palestinesi di Hamas sono come i partigiani"},
    #{"role":"assistant","content":'{"reason":"Il confronto tra i palestinesi di Hamas e i partigiani può essere interpretato come un tentativo di idealizzare o giustificare le azioni di Hamas.", "bias":"no_bias"}uu'},
    {"role":'user', "content":text}]

    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0
    )

    pred_raw.append(response.choices[0].message.content)

  0%|          | 0/28 [00:00<?, ?it/s]

100%|██████████| 28/28 [00:52<00:00,  1.87s/it]


In [38]:
df_binary['pred_label'] = list(map(lambda x: json.loads(x[:-2])['bias'],pred_raw))

In [39]:
df_binary.sample()

,titolo,link,data,giornale,bias_true,pred_label
24,I pro-Palestina assaltano un supermercato: le ...,https://www.liberoquotidiano.it/news/milano/38...,2024-02-24,Libero,pro_israele,pro_israele


In [40]:
preds = df_binary['pred_label'].tolist()
true_labs = df_binary['bias_true'].tolist()

In [ ]:
accuracy_score(true_labs,preds)
#Non buonissimo

0.6428571428571429

# Predizioni per tutti il df

In [27]:
if len(glob.glob('openai_partial.csv'))>0:
    df = pd.read_csv('openai_partial.csv')
    start = len(df.dropna())
    end = len(df)

else:
    df = pd.read_csv('base_csv_for_LLM.csv')
    df['raw_pred'] = [None]*len(df)
    df['bias'] = [None]*len(df) 
    df['reason'] = [None]*len(df)
    start = 0
    end = len(df)

In [20]:
for i in tqdm.tqdm(range(start+1,end)):
    text = df.iloc[i,0]

    messages=[{"role": "system", "content": sistema_binary},
    {"role": "user", "content": "Medio Oriente, Salvini: 'Reazione Israele inevitabile dopo carneficina'"},
    {"role":"assistant","content": '{"reason": "Le azioni di Israele vengono presentate esclusivamente come una reazione giustificata.", "bias":"pro_israele"}uu'},
    {"role": "user", "content": "Saltata la tregua Israele riprende a colpire la Striscia  Ma si tratta ancora"},
    {"role":"assistant", "content":'{"reason": "Il titolo cita entrambe le parti in gioco, senza prendere una posizione e senza nascondere nulla.", "bias":"no_bias"}uu'},
    #{"role":"user", "content":"I palestinesi di Hamas sono come i partigiani"},
    #{"role":"assistant","content":'{"reason":"Il confronto tra i palestinesi di Hamas e i partigiani può essere interpretato come un tentativo di idealizzare o giustificare le azioni di Hamas.", "bias":"no_bias"}uu'},
    {"role":'user', "content":text}]

    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0
    )

    df.iloc[i,4] = response.choices[0].message.content
    df.iloc[i,5] = json.loads(response.choices[0].message.content[:-2])['bias']
    df.iloc[i,6] = json.loads(response.choices[0].message.content[:-2])['reason']

    df.to_csv('openai_partial.csv',index=False) #salviamo a ogni iterazione per evitare problemi

  0%|          | 0/234 [00:00<?, ?it/s]

100%|██████████| 234/234 [07:06<00:00,  1.82s/it]


In [25]:
df.to_csv('openai_def.csv',index=False)
#salvare dataset finale tutto annotato